# 改变颜色空间
- 如何将图像从一个色彩空间转换到另一个，像BGR↔灰色，BGR↔HSV等
- 创建一个应用程序，以提取视频中的彩色对象
- **cv.cvtColor**，**cv.inRange**等。

In [1]:
import cv2 as cv
import numpy as np

## 改变颜色空间
OpenCV中有超过150种颜色空间转换方法。但是我们将研究只有两个最广泛使用的,BGR↔灰色和BGR↔HSV。

对于颜色转换，我们使用cv函数。`cvtColor(input_image, flag)`，其中flag决定转换的类型。

对于BGR→灰度转换，我们使用标志`cv.COLOR_BGR2GRAY`。类似地，对于BGR→HSV，我们使用标志`cv.COLOR_BGR2HSV`。要获取其他标记，只需在Python终端中运行以下命令:

In [2]:
flags = [i for i in dir(cv) if i.startswith('COLOR_')]
print(flags)

['COLOR_BAYER_BG2BGR', 'COLOR_BAYER_BG2BGRA', 'COLOR_BAYER_BG2BGR_EA', 'COLOR_BAYER_BG2BGR_VNG', 'COLOR_BAYER_BG2GRAY', 'COLOR_BAYER_BG2RGB', 'COLOR_BAYER_BG2RGBA', 'COLOR_BAYER_BG2RGB_EA', 'COLOR_BAYER_BG2RGB_VNG', 'COLOR_BAYER_GB2BGR', 'COLOR_BAYER_GB2BGRA', 'COLOR_BAYER_GB2BGR_EA', 'COLOR_BAYER_GB2BGR_VNG', 'COLOR_BAYER_GB2GRAY', 'COLOR_BAYER_GB2RGB', 'COLOR_BAYER_GB2RGBA', 'COLOR_BAYER_GB2RGB_EA', 'COLOR_BAYER_GB2RGB_VNG', 'COLOR_BAYER_GR2BGR', 'COLOR_BAYER_GR2BGRA', 'COLOR_BAYER_GR2BGR_EA', 'COLOR_BAYER_GR2BGR_VNG', 'COLOR_BAYER_GR2GRAY', 'COLOR_BAYER_GR2RGB', 'COLOR_BAYER_GR2RGBA', 'COLOR_BAYER_GR2RGB_EA', 'COLOR_BAYER_GR2RGB_VNG', 'COLOR_BAYER_RG2BGR', 'COLOR_BAYER_RG2BGRA', 'COLOR_BAYER_RG2BGR_EA', 'COLOR_BAYER_RG2BGR_VNG', 'COLOR_BAYER_RG2GRAY', 'COLOR_BAYER_RG2RGB', 'COLOR_BAYER_RG2RGBA', 'COLOR_BAYER_RG2RGB_EA', 'COLOR_BAYER_RG2RGB_VNG', 'COLOR_BGR2BGR555', 'COLOR_BGR2BGR565', 'COLOR_BGR2BGRA', 'COLOR_BGR2GRAY', 'COLOR_BGR2HLS', 'COLOR_BGR2HLS_FULL', 'COLOR_BGR2HSV', 'COLOR_

注意 HSV的色相范围为[0,179]，饱和度范围为[0,255]，值范围为[0,255]。不同的软件使用不同的规模。因此，如果你要将OpenCV值和它们比较，你需要将这些范围标准化。

## 对象追踪
现在我们知道了如何将BGR图像转换成HSV，我们可以使用它来提取一个有颜色的对象。在HSV中比在BGR颜色空间中更容易表示颜色。在我们的应用程序中，我们将尝试提取一个蓝色的对象。方法如下: 
- 取视频的每一帧 
- 转换从BGR到HSV颜色空间 
- 我们对HSV图像设置蓝色范围的阈值 
- 现在单独提取蓝色对象，我们可以对图像做任何我们想做的事情.

### 色调H
用角度度量，取值范围为0°～360°，从红色开始按逆时针方向计算，红色为0°，绿色为120°,蓝色为240°。它们的补色是：黄色为60°，青色为180°,紫色为300°；
### 饱和度S
饱和度S表示颜色接近光谱色的程度。一种颜色，可以看成是某种光谱色与白色混合的结果。其中光谱色所占的比例愈大，颜色接近光谱色的程度就愈高，颜色的饱和度也就愈高。饱和度高，颜色则深而艳。光谱色的白光成分为0，饱和度达到最高。通常取值范围为0%～100%，值越大，颜色越饱和。
### 明度V
明度表示颜色明亮的程度，对于光源色，明度值与发光体的光亮度有关；对于物体色，此值和物体的透射比或反射比有关。通常取值范围为0%（黑）到100%（白）。
RGB和CMY颜色模型都是面向硬件的，而HSV（Hue Saturation Value）颜色模型是面向用户的。

HSV模型的三维表示从RGB立方体演化而来。设想从RGB沿立方体对角线的白色顶点向黑色顶点观察，就可以看到立方体的六边形外形。六边形边界表示色彩，水平轴表示纯度，明度沿垂直轴测量。

In [4]:
cap = cv.VideoCapture(0)
while(1):
    # 读取帧
    _, frame = cap.read()
    # 转换颜色空间 BGR -> HSV
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    # 定义HSV中蓝色的范围
    lower_blue = np.array([110, 50, 50])
    upper_blue = np.array([130, 255, 255])
    #设置HSV阈值使得只取蓝色
    mask = cv.inRange(hsv, lower_blue, upper_blue)
    # mask 和 frame 逐像素相与
    res = cv.bitwise_and(frame, frame, mask=mask)
    cv.imshow('frame', frame)
    cv.imshow('mask', mask)
    cv.imshow('res', res)
    k = cv.waitKey(5) & 0xFF
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()

`inRange()`
Checks if array elements lie between the elements of two other arrays.

The function checks the range as follows:

For every element of a single-channel input array:
$$dst(I)=lowerb(I)_0≤src(I)_0≤upperb(I)_0$$
For two-channel arrays:
$$dst(I)=lowerb(I)+_0≤src(I)_0≤upperb(I)_0∧lowerb(I)_1≤src(I)_1≤upperb(I)_1$$
and so forth.

That is, `dst (I)` is set to `255 (all 1 -bits)` if `src (I)` is within the specified 1D, 2D, 3D, ... box and `0` otherwise.

When the lower and/or upper boundary parameters are scalars, the indexes (I) at lowerb and upperb in the above formulas should be omitted

[![5RJFhj.png](https://z3.ax1x.com/2021/10/23/5RJFhj.png)](https://imgtu.com/i/5RJFhj)

**注意** 图像中有一些噪点。我们将在后面的章节中看到如何删除它们。 这是对象跟踪中最简单的方法。一旦学习了轮廓的功能，你就可以做很多事情，例如找到该对象的质心并使用它来跟踪对象，仅通过将手移到相机前面以及其他许多有趣的东西就可以绘制图表

## 如何找到要追踪的HSV值
这是在stackoverflow.com上发现的一个常见问题。它非常简单，你可以使用相同的函数**cv.cvtColor()**。你只需传递你想要的BGR值，而不是传递图像。例如，要查找绿色的HSV值，请在Python终端中尝试以下命令:

In [6]:
green = np.uint8([[[0, 255, 0]]])
hsv_grenn = cv.cvtColor(green, cv.COLOR_BGR2HSV)
print(hsv_grenn)

[[[ 60 255 255]]]


现在把`[H- 10,100,100]`和`[H+ 10,255, 255]`分别作为下界和上界。除了这个方法之外，你可以使用任何图像编辑工具(如GIMP或任何在线转换器)来查找这些值，但是不要忘记调整HSV范围